In [1]:
import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd
from shapely import geometry
import pandas as pd
import os

# split the county gdf to 4

In [3]:
'''
gdf = gpd.read_file("VA_County_CityFixed2.json")
gdf.head(1)
'''

,County,geometry
0,Accomack County,"POLYGON ((-75.62204 37.45679, -75.62465 37.457..."


In [4]:
'''
counties = sorted(list(gdf["County"].unique()))
print(len(counties))
print(113//4)
'''

133
28


In [5]:
# len(gdf[gdf["County"].isin(counties[3*28:4*28])]["County"].unique())

28

In [6]:
'''
# split to 4 files with 28 counties each file (last file 29)
county_count = 0
for i in range(4):
    if i == 3:
        file_name = "VA" + str(i) + "County.json"
        gdf[gdf["County"].isin(counties[i*28 :])].to_file(file_name, driver="GeoJSON")
        county_count += len(gdf[gdf["County"].isin(counties[i*28 :])])
    else:
        file_name = "VA" + str(i) + "County.json"
        gdf[gdf["County"].isin(counties[i*28 : (i+1)*28])].to_file(file_name, driver="GeoJSON")
        county_count += len(gdf[gdf["County"].isin(counties[i*28 : (i+1)*28])])
print(county_count)
'''

133


# create x y for each polygon

In [7]:
Polygon = geometry.polygon.Polygon
MultiPolygon = geometry.multipolygon.MultiPolygon

In [9]:
def parse(gdf, arg_i, arg_shape_id, arg_polygon_id):
    columns = ["geovertex_id", "shape_id", "polygon_id", "County", "x_pos", "y_pos"]
    df = pd.DataFrame(columns=columns)

    counter = 0
    i = arg_i
    shape_id = arg_shape_id
    polygon_id = arg_polygon_id

    for index, row in gdf.iterrows():
        # for visualization only
        if counter%15==0:
            print(" {:.2f}% ".format(counter/len(gdf)*100), end="")
        else:
            print("#", end="")

        cur_shape = row["geometry"]
        cur_type = type(cur_shape)

        row = row.drop(["geometry"])
        cur_row = row
        cur_row["shape_id"] = shape_id
        shape_id += 1

        if cur_type == MultiPolygon:
            for polygon in cur_shape:
                cur_row["polygon_id"] = polygon_id
                polygon_id += 1
                for coords in polygon.exterior.coords:
                    cur_row["geovertex_id"] = i
                    i += 1
                    cur_row["x_pos"] = coords[0]
                    cur_row["y_pos"] = coords[1]
                    df = df.append(cur_row, ignore_index=True)
                    cur_row = row
        elif cur_type == Polygon:
            cur_row["polygon_id"] = polygon_id
            polygon_id += 1
            for coords in cur_shape.exterior.coords:
                cur_row["geovertex_id"] = i
                i += 1
                cur_row["x_pos"] = coords[0]
                cur_row["y_pos"] = coords[1]
                df = df.append(cur_row, ignore_index=True)
                cur_row = row
        counter += 1
        
    return df

In [10]:
next_i, next_shape, next_poly = 2990000, 1, 3995
for i in range(4):
    file_name = "VA" + str(i) + "County.json"
    
    print("\n\nStart", file_name)
    gdf = gpd.read_file(file_name)
    gdf_Parsed = parse(gdf, arg_i=next_i, arg_shape_id=next_shape, arg_polygon_id=next_poly)

    file_name = file_name.split(".")[0]
    gdf_Parsed.to_csv(file_name+"_Parsed.csv")

    next_i = gdf_Parsed.iloc[-1]["geovertex_id"] + 1
    next_shape = gdf_Parsed.iloc[-1]["shape_id"] + 1
    next_poly = gdf_Parsed.iloc[-1]["polygon_id"] + 1
    print("\nLast row id: geo:{}, shape:{}, poly:{}".format(gdf_Parsed.iloc[-1]["geovertex_id"],
                                                         gdf_Parsed.iloc[-1]["shape_id"],
                                                         gdf_Parsed.iloc[-1]["polygon_id"]))
    print("Next id: geo:{}, shape:{}, poly:{}".format(next_i, next_shape, next_poly))



Start VA0County.json
 0.00% #
Last row id: geo:2991066, shape:1, poly:3995
Next id: geo:2991067, shape:2, poly:3996


In [14]:
all_dfs = []
rows = 0
for i in range(4):
    FILE_NAME = "VA" + str(i) + "County_Parsed.csv"
    all_dfs.append(pd.read_csv(FILE_NAME))
    rows += len(all_dfs[-1])

merged = pd.concat(all_dfs)
print(len(merged), rows)

557307 557307


In [15]:
merged.to_csv("VA_County_Parsed_Merged.csv")

In [4]:
#precinct = pd.read_csv("../precinct/VA_Final_Merged.csv")

In [5]:
#precinct.head(-1)

,geovertex_id,shape_id,polygon_id,x_pos,y_pos
0,419895,1,941,-76.943785,37.780534
1,419896,1,941,-76.944097,37.780237
2,419897,1,941,-76.944536,37.779986
3,419898,1,941,-76.945793,37.779563
4,419899,1,941,-76.946149,37.779332
...,...,...,...,...,...
2569558,2989453,2446,3467,-76.480168,37.236542
2569559,2989454,2446,3467,-76.479523,37.236377
2569560,2989455,2446,3467,-76.478359,37.236079
2569561,2989456,2446,3467,-76.473780,37.234929


# fix county index, this should be after precinct

In [11]:
#county = pd.read_csv("VA_County_Parsed_Merged.csv")

In [12]:
#county.head()

,geovertex_id,shape_id,polygon_id,County,x_pos,y_pos
0,2989459,1,3468,Accomack County,-75.622044,37.456787
1,2989460,1,3468,Accomack County,-75.624647,37.457400
2,2989461,1,3468,Accomack County,-75.630694,37.458797
3,2989462,1,3468,Accomack County,-75.655216,37.464464
4,2989463,1,3468,Accomack County,-75.661913,37.466192


In [8]:
#county.drop(["Unnamed: 0", "Unnamed: 0.1"], inplace=True, axis=1)
#county.head(1) 

13082

In [13]:
'''
geo = 7487412
poly = 9614
counter = 0
for index, row in county.iterrows():
    # for visualization only
    if counter%50000==0:
        print(" {:.2f}% ".format(counter/len(county)*100), end="")
    elif counter%1000==0:
        print("#", end="")
    
    
    county.at[index, "geovertex_id"] = geo
    geo += 1
    county.at[index, "polygon_id"] += poly
    counter += 1
    
county.head()
'''

 0.00% ################################################# 8.97% ################################################# 17.94% ################################################# 26.92% ################################################# 35.89% ################################################# 44.86% ################################################# 53.83% ################################################# 62.80% ################################################# 71.77% ################################################# 80.75% ################################################# 89.72% ################################################# 98.69% #######

,geovertex_id,shape_id,polygon_id,County,x_pos,y_pos
0,7487412,1,13082,Accomack County,-75.622044,37.456787
1,7487413,1,13082,Accomack County,-75.624647,37.457400
2,7487414,1,13082,Accomack County,-75.630694,37.458797
3,7487415,1,13082,Accomack County,-75.655216,37.464464
4,7487416,1,13082,Accomack County,-75.661913,37.466192


In [14]:
# county.to_csv("VA_County_Parsed_Merged.csv", index=False)